In [1]:
# 1. imports
import pandas as pd
import numpy as np
from datetime import datetime

# optional: display settings
pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
# 2. real 2025 highlights for TMICC (Magnum-led business)
financial_highlights = pd.DataFrame({
    'year': [2025],
    'currency': ['EUR'],
    'revenue_billion': [7.9],
    'organic_sales_growth_pct': [4.2],
    'operating_profit_million': [599],
    'adj_ebitda_million': [1255],
    'adj_ebit_million': [917],
    'op_profit_margin_pct': [7.6],
    'adj_ebitda_margin_pct': [15.9],
    'adj_ebit_margin_pct': [11.6]
})

financial_highlights

,year,currency,revenue_billion,organic_sales_growth_pct,operating_profit_million,adj_ebitda_million,adj_ebit_million,op_profit_margin_pct,adj_ebitda_margin_pct,adj_ebit_margin_pct
0,2025,EUR,7.90,4.20,599,1255,917,7.60,15.90,11.60


In [3]:
# 3a. regions
regions = ['Europe_ANZ', 'Americas', 'AMEA']
# illustrative revenue shares (must sum to 1.0 for whole revenue)
# These are example splits; adjust or replace with real data as available
shares = [0.40, 0.35, 0.25]

annual_region_sales = pd.DataFrame({
    'year': [2025]*len(regions),
    'region': regions,
    'rev_share': shares
})

# convert shares to revenue amounts
overall_rev = financial_highlights.loc[0, 'revenue_billion'] * 1_000  # in million EUR
annual_region_sales['revenue_million'] = annual_region_sales['rev_share'] * overall_rev

annual_region_sales

,year,region,rev_share,revenue_million
0,2025,Europe_ANZ,0.40,"3,160.00"
1,2025,Americas,0.35,"2,765.00"
2,2025,AMEA,0.25,"1,975.00"


In [4]:
# 3b. add sample margins and growth
# seed from official lines: Europe & ANZ and Americas margin snippets
# margin examples are illustrative; adjust if actual figures differ
region_details = {
    'Europe_ANZ': {'agg_margin_pct': 13.1, 'organic_sales_growth_pct': 3.3},
    'Americas': {'agg_margin_pct': 14.1, 'organic_sales_growth_pct': 0.8},
    'AMEA': {'agg_margin_pct': 22.9, 'organic_sales_growth_pct': 10.9}
}

def lookup_region_val(region, key):
    return region_details.get(region, {}).get(key, np.nan)

annual_region_sales['adj_ebitda_margin_pct'] = annual_region_sales['region'].map(
    lambda r: lookup_region_val(r, 'agg_margin_pct')
)
annual_region_sales['organic_sales_growth_pct'] = annual_region_sales['region'].map(
    lambda r: lookup_region_val(r, 'organic_sales_growth_pct')
)

annual_region_sales

,year,region,rev_share,revenue_million,adj_ebitda_margin_pct,organic_sales_growth_pct
0,2025,Europe_ANZ,0.40,"3,160.00",13.10,3.30
1,2025,Americas,0.35,"2,765.00",14.10,0.80
2,2025,AMEA,0.25,"1,975.00",22.90,10.90


In [5]:
# 4. generate quarterly sample for 2025
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
q_df = pd.DataFrame([
    {'year': 2025, 'quarter': q} for q in quarters
])

# simple split assuming even quarter weights; adjust per real seasonality if known
q_df['rev_million'] = overall_rev / len(quarters)

# add random small noise to simulate variation
np.random.seed(42)
noise = np.random.normal(loc=0, scale=0.05, size=len(q_df))  # +/-5% variation
q_df['rev_million_noisy'] = q_df['rev_million'] * (1 + noise)

# sample margin as overall adjusted EBITDA margin
overall_margin = financial_highlights.loc[0, 'adj_ebitda_margin_pct'] / 100
q_df['ebitda_million_est'] = q_df['rev_million_noisy'] * overall_margin

q_df

,year,quarter,rev_million,rev_million_noisy,ebitda_million_est
0,2025,Q1,"1,975.00","2,024.05",321.82
1,2025,Q2,"1,975.00","1,961.35",311.85
2,2025,Q3,"1,975.00","2,038.96",324.19
3,2025,Q4,"1,975.00","2,125.40",337.94


In [6]:
# 5a. demographic mock data
age_groups = ['18-24', '25-34', '35-44', '45-54', '55+']

# assume for each region a distribution that sums to 1
demog_list = []
for region in regions:
    # random distribution; ensure sums to 1
    dist = np.random.dirichlet(np.ones(len(age_groups)), size=1).flatten()
    for age, share in zip(age_groups, dist):
        demog_list.append({
            'region': region,
            'age_group': age,
            'share_of_consumers': share
        })

demog_df = pd.DataFrame(demog_list)
demog_df

,region,age_group,share_of_consumers
0,Europe_ANZ,18-24,0.05
1,Europe_ANZ,25-34,0.05
2,Europe_ANZ,35-44,0.02
3,Europe_ANZ,45-54,0.60
4,Europe_ANZ,55+,0.28
5,Americas,18-24,0.18
6,Americas,25-34,0.00
7,Americas,35-44,0.52
8,Americas,45-54,0.26
9,Americas,55+,0.04


In [7]:
# 5b. simulate average annual spend per age group (EUR)
# simple illustrative values; adjust to realistic levels
avg_spend_map = {'18-24': 25, '25-34': 35, '35-44': 40, '45-54': 30, '55+': 20}

demog_df['avg_annual_spend_eur'] = demog_df['age_group'].map(avg_spend_map)
demog_df

,region,age_group,share_of_consumers,avg_annual_spend_eur
0,Europe_ANZ,18-24,0.05,25
1,Europe_ANZ,25-34,0.05,35
2,Europe_ANZ,35-44,0.02,40
3,Europe_ANZ,45-54,0.60,30
4,Europe_ANZ,55+,0.28,20
5,Americas,18-24,0.18,25
6,Americas,25-34,0.00,35
7,Americas,35-44,0.52,40
8,Americas,45-54,0.26,30
9,Americas,55+,0.04,20


In [8]:
# 6. combine region revenue with demographic shares
# for demonstration, compute revenue by region and age group (rough estimate)
# This multiplies region revenue by share of consumers, then by average spend ratio
# NOTE: purely illustrative

# first, merge region revenue into demog_df
demog_with_rev = demog_df.merge(
    annual_region_sales[['region', 'revenue_million']],
    on='region',
    how='left'
)

# estimate revenue contribution per demographic slice
# assume demographic share reflects share of consumers; scale by region revenue
demog_with_rev['estimated_rev_million'] = demog_with_rev['share_of_consumers'] * demog_with_rev['revenue_million']

demog_with_rev.head(10)

,region,age_group,share_of_consumers,avg_annual_spend_eur,revenue_million,estimated_rev_million
0,Europe_ANZ,18-24,0.05,25,"3,160.00",161.00
1,Europe_ANZ,25-34,0.05,35,"3,160.00",160.97
2,Europe_ANZ,35-44,0.02,40,"3,160.00",56.79
3,Europe_ANZ,45-54,0.60,30,"3,160.00","1,908.91"
4,Europe_ANZ,55+,0.28,20,"3,160.00",872.33
5,Americas,18-24,0.18,25,"2,765.00",502.07
6,Americas,25-34,0.00,35,"2,765.00",8.48
7,Americas,35-44,0.52,40,"2,765.00","1,428.66"
8,Americas,45-54,0.26,30,"2,765.00",728.46
9,Americas,55+,0.04,20,"2,765.00",97.33


In [9]:
# 7. export to CSVs for external use
financial_highlights.to_csv('financial_highlights_2025.csv', index=False)
annual_region_sales.to_csv('region_sales_2025.csv', index=False)
q_df.to_csv('quarterly_sales_2025.csv', index=False)
demog_df.to_csv('demographics_mock.csv', index=False)
demog_with_rev.to_csv('demographics_with_estimated_rev.csv', index=False)